In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
#mae(ts_data[['cleanliness','location','rooms','service','value']], pred_test_data )

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('hamming')
mae_hist = []
mae_cleanliness = []
mae_loc = []
mae_rooms = []
mae_srv = []

mae_value = []

rmse_hist = []
rmse_cleanliness = []
rmse_loc = []
rmse_rooms = []
rmse_srv = []

rmse_value = []
k= 7
f = 1

for i in range(1,6):
    sm = 0
    train_data = pd.read_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment'+str(i)+'_train.csv')
    y_train  =train_data[['cleanliness','location','rooms','service','value']].values
    test_data = pd.read_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment'+str(i)+'_test.csv')
    y_test = test_data[['cleanliness','location','rooms','service','value']].values
    pred_Bert_cf = []
    print('fold',i)
    cmp=0
    for idx,row in test_data.iterrows():
      cmp+=1
      precedent_data = train_data[train_data['user_id']==row['user_id']]
      precedent_hotels = precedent_data.offering_id.values
      prcd_reviews = precedent_data.text.values
      hotel_id = row['offering_id']
      
      if hotel_id in train_data.offering_id.values:
        precedent_data = train_data[train_data['user_id']==row['user_id']]

        if precedent_data.shape[0] > 0:

          precedent_hotels = precedent_data.offering_id.values
          prcd_reviews = precedent_data[['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values
          
          
        
          prcedent_data_sentiment_review = {}
          for h_id,rev in zip(precedent_hotels,prcd_reviews):
            
            
            prcedent_data_sentiment_review[h_id] = rev
            
          
          user = row['user_id']

          maybe_sim_users = []
          sim = {}
          if cmp%100==0:
            print('rest',test_data.shape[0] - cmp)

          users = train_data[train_data['offering_id']==hotel_id].user_id.unique()
          for user1 in users:
            if (user1 != user):
              

              user1_data = train_data[train_data['user_id']==user1]
              sim_items = set(precedent_hotels).intersection(set(user1_data.offering_id.values))
              if len(sim_items) > 1:
                  
                  maybe_sim_users.append(user1)
                
                  user_rat = []
                  user1_rat = []
                  for item in sim_items:
                    for j in range(5):
                      user_rat.append(prcedent_data_sentiment_review[item][j])

                  
                    rev = user1_data[user1_data['offering_id']==item][['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values
                    
                    
                    
                    for j in range(5):
                      user1_rat.append(rev[0][j])
                  
                  if  1 - abs(dist.pairwise([user_rat],[user1_rat])[0][0]) > 0.4:        
                    sim[user1] = 1 - abs(dist.pairwise([user_rat],[user1_rat])[0][0])

                    
                  

          
          if len(sim.keys())>1: #similar users
            sm+=1
            
            rat = [0.0,0.0,0.0,0.0,0.0]
            rat_mul = [0.0,0.0,0.0,0.0,0.0]
            sum_sim = 0.0
            #for user1 in list(sim.keys())[:k]:
            #rat += sim[user1] * train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]
            #sum_sim+= sim[user1]
            #rat = rat / sum_sim

            for user1 in list(sim.keys())[:k]:
              user1_data = train_data[train_data['user_id']==user1]
              rev = train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][
                                            ['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values[0]
              rat_mul =  train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]


              #rat = rat+ sim[user1]*0.5*(rev*5+rat_mul) 
              rat += rat_mul*sim[user1]
              sum_sim += sim[user1]
            rat = rat / sum_sim
            
            
          
          
          else : #no similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)
          
        else: #new_user
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)  

      elif train_data[train_data['user_id']==row['user_id']].shape[0]>0: #new_item
        rat = [0.0,0.0,0.0,0.0,0.0]
        for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
            rat[i] = np.mean(precedent_data[[col]].values)
      else: #new_user , new_item
        #rnd_petubation = random.uniform(-1,1)
        #val = rnd_petubation + 3.0
        rat = [train_data['cleanliness'].mean(),train_data['location'].mean(),train_data['rooms'].mean()
            ,train_data['service'],train_data['value']]
      pred_Bert_cf.append(rat.copy())
    print(sm)

    pred_Bert_cf = np.array(pred_Bert_cf)
    pred_test_data = np.copy(pred_Bert_cf)
    mae_cleanliness.append(mae(pred_test_data[:,0],y_test[:,0]))
    mae_loc.append(mae(pred_test_data[:,1],y_test[:,1]))
    mae_rooms.append(mae(pred_test_data[:,2],y_test[:,2]))
    mae_srv.append(mae(pred_test_data[:,3],y_test[:,3]))
    
    mae_value.append(mae(pred_test_data[:,4],y_test[:,4]))
    mae_hist.append(mae(pred_test_data,y_test))
      
    rmse_cleanliness.append(mse(pred_test_data[:,0],y_test[:,0], squared=False))
    rmse_loc.append(mse(pred_test_data[:,1],y_test[:,1], squared=False))
    rmse_rooms.append(mse(pred_test_data[:,2],y_test[:,2], squared=False))
    rmse_srv.append(mse(pred_test_data[:,3],y_test[:,3], squared=False))
  
    rmse_value.append(mse(pred_test_data[:,4],y_test[:,4], squared=False))
    rmse_hist.append(mse(pred_test_data,y_test, squared=False))

    print(mae_hist)
    print(rmse_hist)

fold 1
rest 1834
rest 1734
rest 1634
rest 1534
rest 1434
rest 1334
rest 1234
rest 1134
rest 1034
rest 934
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
49
[0.8226052413632015]
[1.1117381223380998]
fold 2
rest 1834
rest 1734
rest 1634
rest 1534
rest 1334
rest 1234
rest 1134
rest 1034
rest 934
rest 834
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
43
[0.8226052413632015, 0.8107435568262735]
[1.1117381223380998, 1.0998562886736136]
fold 3
rest 1834
rest 1734
rest 1634
rest 1534
rest 1434
rest 1334
rest 1234
rest 1134
rest 1034
rest 834
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
43
[0.8226052413632015, 0.8107435568262735, 0.8180883235349476]
[1.1117381223380998, 1.0998562886736136, 1.1085063192708795]
fold 4
rest 1833
rest 1733
rest 1633
rest 1533
rest 1433
rest 1333
rest 1233
rest 1133
rest 1033
rest 933
rest 833
rest 733
rest 633
rest 533
rest 433
rest 333
rest 233
rest 133
rest 33
41
[0.8226052413632015, 0.810

In [ ]:
print('mean_mae',mae_hist , np.mean(mae_hist))
print('mae_cleanliness',mae_cleanliness ,np.mean(mae_cleanliness ))
print('mae_loc',mae_loc,np.mean(mae_loc))
print('mae_rooms',mae_rooms,np.mean(mae_rooms))
print('mae_srv',mae_srv,np.mean(mae_srv))

print('mae_value',mae_value,np.mean(mae_value))

print('rmse_hist',rmse_hist,np.mean(rmse_hist))
print('rmse_cleanliness',rmse_cleanliness,np.mean(rmse_cleanliness))
print('rmse_loc',rmse_loc,np.mean(rmse_loc))
print('rmse_rooms',rmse_rooms,np.mean(rmse_rooms))
print('rmse_srv',rmse_srv,np.mean(rmse_srv))

print('rmse_value',rmse_value,np.mean(rmse_value))

mean_mae [0.8226052413632015, 0.8107435568262735, 0.8180883235349476, 0.815524141893856, 0.8259155119836048] 0.8185753551203767
mae_cleanliness [0.7618216715804095, 0.7581670096775832, 0.7416879233404235, 0.735791160844956, 0.7779946963587561] 0.7550924923604256
mae_loc [0.6450273501400449, 0.6450543318850589, 0.6215113659636217, 0.6437018609774574, 0.6278856158355014] 0.6366361049603368
mae_rooms [0.8598148738680219, 0.8549221051317087, 0.8691231329772916, 0.8624612443591615, 0.8507756712451403] 0.8594194055162647
mae_srv [0.8849328240526754, 0.8624403479070674, 0.8989181381658022, 0.8773008665897704, 0.905005388048318] 0.8857195129527267
mae_value [0.9614294871748567, 0.9331339895299474, 0.9592010572276045, 0.9583655766979332, 0.9679161884303122] 0.9560092598121308
rmse_hist [1.1117381223380998, 1.0998562886736136, 1.1085063192708795, 1.0965271167806878, 1.1122314590792506] 1.1057718612285061
rmse_cleanliness [1.0579356889565228, 1.0627161936480234, 1.034498483278117, 1.0252299691794

In [ ]:
test_data

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,...,date_stayed,offering_id,date,via_mobile,user_id,cleanliness_sent,location_sent,rooms_sent,service_sent,value_sent
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,stayed us andaz properties one best much bette...,JamesE339,34.0,...,December 2012,1927.0,"December 17, 2012",False,626.0,1,1,1,1,1
1,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,booked hotel thinking another hilton property ...,beatricepugzilla,28.0,...,March 2012,849.0,"April 1, 2012",False,583.0,1,1,1,1,1
2,4.0,4.0,4.0,4.0,4.0,4.0,“Fine hotel but no executive level”,stayed one night business trip hotel fine clea...,Member3,29.0,...,February 2012,849.0,"February 15, 2012",False,658.0,1,1,1,1,1
3,5.0,5.0,5.0,5.0,5.0,4.0,“Weel worth it”,even though live houston decided take mini-vac...,WardW_12,11.0,...,June 2012,1799.0,"July 2, 2012",False,77.0,1,1,0,1,1
4,4.0,5.0,5.0,4.0,5.0,5.0,“Great location”,hilton americas houston large hotel downtown h...,Anthony B,9.0,...,April 2012,1506.0,"October 4, 2012",False,590.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,3.0,4.0,1.0,3.0,4.0,4.0,“A long hike to the action”,good value hotel major challenges location vie...,asiandude,26.0,...,July 2010,1449.0,"July 31, 2010",False,526.0,1,1,1,1,1
1929,4.0,5.0,5.0,5.0,5.0,4.0,"“Great service, lovely room but request room a...",traveled boston daughter could tour interview ...,jenifoo,NaN,...,February 2010,1449.0,"February 20, 2010",False,432.0,1,1,1,1,1
1930,5.0,4.0,5.0,5.0,5.0,4.0,“Hotel Palomar exceeded our expectations”,initially booked hotel since reviews good loca...,love2run206,2.0,...,October 2010,353.0,"November 3, 2010",False,10.0,1,1,1,1,1
1931,4.0,4.0,4.0,4.0,3.0,2.0,"“huge space, nice place”",decent breakfast large lounge enjoy didnt time...,keeevm,5.0,...,March 2010,298.0,"March 8, 2010",False,811.0,1,1,1,1,1
